In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/jay-image/jay.jpg
/kaggle/input/gvideo/input.mp4
/kaggle/input/video-test/input.mp4
/kaggle/input/yolowieghts/yolo11n.pt
/kaggle/input/yolo8nw/yolov8n.pt


In [3]:
!mkdir known_faces

In [1]:
!cp /kaggle/input/gvideo/input.mp4 /kaggle/working/

In [4]:
!cp /kaggle/input/jay-image/jay.jpg /kaggle/working/known_faces

In [8]:
import cv2
from ultralytics import YOLO
from deepface import DeepFace
import os

# Load the YOLOv8 model
model = YOLO('/kaggle/input/yolo8nw/yolov8n.pt')  # 'yolov8n.pt' is a small version for testing; you can use larger models if you have a GPU.

# Path to the directory with known faces
known_faces_dir = 'known_faces/'
known_faces = {}

# Load known faces and their names into memory
for file in os.listdir(known_faces_dir):
    if file.endswith(('jpg', 'jpeg', 'png')):
        name = file.split('.')[0]  # Assuming filenames are person names
        image_path = os.path.join(known_faces_dir, file)
        known_faces[name] = DeepFace.represent(image_path, enforce_detection=False)

# Function to recognize and name a face
def recognize_face(frame, bbox):
    x1, y1, x2, y2 = bbox
    face = frame[y1:y2, x1:x2]

    try:
        # Perform face recognition using DeepFace
        result = DeepFace.find(face, db_path=known_faces_dir, enforce_detection=False)
        if result and len(result) > 0:
            # If a match is found, get the name of the recognized person
            matched_face = result[0]
            label = matched_face.split('/')[-1].split('.')[0]
        else:
            label = "Unknown"
    except:
        label = "Unknown"
    
    return label

# Initialize video capture
input_video = '/kaggle/input/gvideo/input.mp4'  # Replace with your input video path
output_video = '/kaggle/working/output.mp4'  # Output video path
cap = cv2.VideoCapture(input_video)

# Get video properties
fps = int(cap.get(cv2.CAP_PROP_FPS))
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

# Initialize video writer to save output
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter(output_video, fourcc, fps, (width, height))

# Process the video frame by frame
while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    # Run YOLO model to detect objects in the frame
    results = model(frame)

    for result in results:
        for box in result.boxes:
            x1, y1, x2, y2 = map(int, box.xyxy[0])  # Bounding box coordinates
            class_id = int(box.cls)
            confidence = box.conf
            class_name = model.names[class_id]

            # Draw a bounding box for each detected object
            cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
            label = f"{class_name}: {float(confidence):.2f}"
            cv2.putText(frame, label, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

            # If the detected object is a person, recognize their face
            if class_name == 'person':
                face_label = recognize_face(frame, (x1, y1, x2, y2))
                cv2.putText(frame, face_label, (x1, y2 + 20), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 0, 0), 2)

    # Write the processed frame to the output video
    out.write(frame)

    # Display the frame (optional)
   # cv2.imshow('Frame', frame)
 #   if cv2.waitKey(1) & 0xFF == ord('q'):
 #       break

# Release the video capture and writer
cap.release()
out.release()
cv2.destroyAllWindows()
            

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.
24-11-08 06:14:23 - Directory /root/.deepface has been created
24-11-08 06:14:23 - Directory /root/.deepface/weights has been created
24-11-08 06:14:24 - vgg_face_weights.h5 will be downloaded...


Downloading...
From: https://github.com/serengil/deepface_models/releases/download/v1.0/vgg_face_weights.h5
To: /root/.deepface/weights/vgg_face_weights.h5
100%|██████████| 580M/580M [00:01<00:00, 423MB/s] 



0: 384x640 3 persons, 1 cup, 4 chairs, 1 potted plant, 2 dining tables, 1 laptop, 72.1ms
Speed: 6.1ms preprocess, 72.1ms inference, 241.5ms postprocess per image at shape (1, 3, 384, 640)
24-11-08 06:14:31 - Found 1 newly added image(s), 0 removed image(s), 0 replaced image(s).


Finding representations: 100%|██████████| 1/1 [00:00<00:00,  1.94it/s]


24-11-08 06:14:32 - There are now 1 representations in ds_model_vggface_detector_opencv_aligned_normalization_base_expand_0.pkl
24-11-08 06:14:32 - Searching [[[  0 255   0]
  [  0 255   0]
  [  0 255   0]
  ...
  [  0 255   0]
  [  0 255   0]
  [  0 255   0]]

 [[  0 255   0]
  [  0 255   0]
  [  0 255   0]
  ...
  [  0 255   0]
  [  0 255   0]
  [  0 255   0]]

 [[  0 255   0]
  [  0 255   0]
  [ 50 121  87]
  ...
  [ 31  29  41]
  [ 30  28  40]
  [  0 255   0]]

 ...

 [[  0 255   0]
  [  0 255   0]
  [ 39  52  94]
  ...
  [ 16   1   2]
  [ 16   1   2]
  [  0 255   0]]

 [[  0 255   0]
  [  0 255   0]
  [ 52  71 114]
  ...
  [ 16   1   2]
  [ 16   1   2]
  [  0 255   0]]

 [[  0 255   0]
  [  0 255   0]
  [  0 255   0]
  ...
  [  0 255   0]
  [  0 255   0]
  [  0 255   0]]] in 1 length datastore
24-11-08 06:14:32 - find function duration 0.8801126480102539 seconds
24-11-08 06:14:32 - Searching [[[  0 255   0]
  [  0 255   0]
  [  0 255   0]
  ...
  [  0 255   0]
  [  0 255   0]
  [ 

error: OpenCV(4.10.0) /io/opencv/modules/highgui/src/window.cpp:1295: error: (-2:Unspecified error) The function is not implemented. Rebuild the library with Windows, GTK+ 2.x or Cocoa support. If you are on Ubuntu or Debian, install libgtk2.0-dev and pkg-config, then re-run cmake or configure script in function 'cvDestroyAllWindows'


In [6]:
!pip install ultralytics

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 881.2/881.2 kB 19.1 MB/s eta 0:00:0000:01


In [7]:
!pip install deepface

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 87.2/87.2 kB 4.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 108.6/108.6 kB 7.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.0/85.0 kB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 42.8 MB/s eta 0:00:0000:01
  Created wheel for fire: filename=fire-0.7.0-py3-none-any.whl size=114248 sha256=e6d1b6732b084d802c42860f6918cc9126b2f6cfbb04370961123953aff922cc
  Stored in directory: /root/.cache/pip/wheels/19/39/2f/2d3cadc408a8804103f1c34ddd4b9f6a93497b11fa96fe738e
Successfully built fire
